<h1 align="center">MSIN0114: Business Analytics Consulting Project</h1>
<h2 align="center">S2R Analytics, pt. 3.1</h2>

# Table of Contents

* [Part 6](#part6): Classification
    * [6.0](#6_0): Data splitting
    * [6.1](#6_1): Models
    * [6.2](#6_2): Models comparison
<br />
<br />
* [Part 7](#part7): Fine-tuning
    * [7.1](#7_1): XGBoost grid search
    * [7.2](#7_2): Random forest classifier grid earch
<br />
<br />
* [Part 8](#part8): Ensemble learning
    * [8.1](#8_1): Voting classifier
    * [8.2](#8_2): Stacking
<br />
<br />
* [Part 9](#part9): Evaluation of the final model
    * [9.1](#9_1): Confusion matrix
    * [9.2](#9_2): ROC curve
    * [9.3](#9_3): Prediction-recall curve
    * [9.4](#9_4): Cost matrix
    * [9.5](#9_5): Feature importance

## Notebook Setup

In [1]:
# Essentials
import pandas as pd
from pandas import Series, DataFrame
from pandas.api.types import CategoricalDtype
pd.options.display.max_columns = None
import sqlite3
import pyodbc
import numpy as np; np.random.seed(1)

# Image creation and display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
from matplotlib import pyplot
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.ticker import FuncFormatter
from yellowbrick.model_selection import FeatureImportances

# Preprocessing
from sklearn import preprocessing
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.decomposition import PCA

# Metrics of accuracy
from numpy import mean
from numpy import std
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from pycm import *
import imbalanced_ensemble as imbens
from imbalanced_ensemble.ensemble.base import sort_dict_by_key
from collections import Counter

# Fine-tuning and enseble learning
from pprint import pprint
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV

#- Other
import itertools as it
import io
import os
os.sys.path
import sys
import glob
import concurrent.futures
from __future__ import print_function
import binascii
import struct
from PIL import Image
import scipy
import scipy.misc
import scipy.cluster
import datetime, time
import functools, operator
from datetime import datetime
from numpy.random import seed
from numpy.random import randn
from numpy import percentile

In [2]:
df = pd.read_csv('csv-files/resampled_compact_data.csv')

## Part 6: <a class="anchor" id="part6"></a> Classification

### 6.0 <a class="anchor" id="6_0"></a> Data splitting

In [3]:
# Choose dependent variables
Y = df[['Rec_Class']]

# Drop the dependent variables from the feature data set
X = df.drop(columns = ['Rec_Class', 'Profit_Class'])

# Scale the explanatory variables
X1 = pd.DataFrame(preprocessing.normalize(X))
X1.columns = X.columns
X = X1

# Split data set into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1, stratify = Y)

print(f'No. of training data: {X_train.shape[0]}')
print(f'No. of training targets: {Y_train.shape[0]}')
print(f'No. of testing data: {X_test.shape[0]}')
print(f'No. of testing targets: {Y_test.shape[0]}')

No. of training data: 7437
No. of training targets: 7437
No. of testing data: 1860
No. of testing targets: 1860


### 6.1 <a class="anchor" id="6_1"></a> Models

#### 6.1.1  <a class="anchor" id="6_1_1"></a> Logistic Regression

In [4]:
# Create a logistic regression classifier model
log = LogisticRegression(random_state = 1, max_iter = 30000)

# Train the model using train set
log.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
log_y_pred=log.predict(X_test)

# Accuracy measures
print('Accuracy score of LOG: ' + str(round(metrics.accuracy_score(Y_test, np.round(log_y_pred)), 3)*100)+'%')
print('Precision score of LOG: ' + str(round(metrics.precision_score(Y_test, np.round(log_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('Recall score of LOG: ' + str(round(metrics.recall_score(Y_test, np.round(log_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('F1 of LOG: ' + str(round(metrics.f1_score(Y_test, np.round(log_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of LOG: 68.30000000000001%
Precision score of LOG: 68.30000000000001%
Recall score of LOG: 68.30000000000001%
F1 of LOG: 68.30000000000001%


#### 6.1.2  <a class="anchor" id="6_1_2"></a> Ridge regression classifier

In [5]:
# Create a ridge regression classifier model
rdg = RidgeClassifier(alpha=1.0, random_state = 1, max_iter = 30000)

# Train the model using train set
rdg.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
rdg_y_pred=rdg.predict(X_test)

# Accuracy measures
print('Accuracy score of RDG: ' + str(round(metrics.accuracy_score(Y_test, np.round(rdg_y_pred)), 3)*100)+'%')
print('Precision score of RDG: ' + str(round(metrics.precision_score(Y_test, np.round(rdg_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('Recall score of RDG: ' + str(round(metrics.recall_score(Y_test, np.round(rdg_y_pred), average='weighted', zero_division=0), 3)*100)+'%')
print('F1 of RDG: ' + str(round(metrics.f1_score(Y_test, np.round(rdg_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of RDG: 68.10000000000001%
Precision score of RDG: 68.10000000000001%
Recall score of RDG: 68.10000000000001%
F1 of RDG: 68.10000000000001%


#### 6.1.3 <a class="anchor" id="6_1_3"></a> K-Neighbours classifier

In [6]:
# Create a k-Neighbours classifier model with 100 neighbours
np.random.seed(1)
knn_100 = KNeighborsClassifier(n_neighbors=100)

# Train the model using train set
knn_100.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
knn_100_y_pred = knn_100.predict(X_test)

# Accuracy measures
print('Accuracy score of KNN-100: ' + str(round(metrics.accuracy_score(Y_test, np.round(knn_100_y_pred)), 3)*100)+'%')
print('Precision score of KNN-100: ' + str(round(metrics.precision_score(Y_test, np.round(knn_100_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of KNN-100: ' + str(round(metrics.recall_score(Y_test, np.round(knn_100_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of KNN-100: ' + str(round(metrics.f1_score(Y_test, np.round(knn_100_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of KNN-100: 66.0%
Precision score of KNN-100: 66.10000000000001%
Recall score of KNN-100: 66.0%
F1 of KNN-100: 66.0%


#### 6.1.4  <a class="anchor" id="6_1_4"></a> Decision tree classifier

In [7]:
# Create decision tree classifier model
dtc = DecisionTreeClassifier(random_state = 1)

# Train the model using train set
dtc = dtc.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
dtc_y_pred = dtc.predict(X_test)

# Accuracy measures
print('Accuracy score of DTC: ' + str(round(metrics.accuracy_score(Y_test, np.round(dtc_y_pred)), 3)*100)+'%')
print('Precision score of DTC: ' + str(round(metrics.precision_score(Y_test, np.round(dtc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of DTC: ' + str(round(metrics.recall_score(Y_test, np.round(dtc_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of DTC: ' + str(round(metrics.f1_score(Y_test, np.round(dtc_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of DTC: 63.6%
Precision score of DTC: 63.6%
Recall score of DTC: 63.6%
F1 of DTC: 63.6%


#### 6.1.5  <a class="anchor" id="6_1_5"></a> Random forest classifier

In [8]:
# Create a random forest classifier model
rfc = RandomForestClassifier(random_state = 1)

# Train the model using train set
rfc.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
rfc_y_pred=rfc.predict(X_test)

# Accuracy measures
print('Accuracy score of RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(rfc_y_pred)), 3)*100)+'%')
print('Precision score of RFC: ' + str(round(metrics.precision_score(Y_test, np.round(rfc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of RFC: ' + str(round(metrics.recall_score(Y_test, np.round(rfc_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of RFC: ' + str(round(metrics.f1_score(Y_test, np.round(rfc_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of RFC: 70.3%
Precision score of RFC: 70.39999999999999%
Recall score of RFC: 70.3%
F1 of RFC: 70.3%


#### 6.1.6  <a class="anchor" id="6_1_6"></a> Gaussian classifier

In [9]:
# Create a Gaussian classifier model
xgbc = XGBClassifier(n_estimators=100, learning_rate=0.05, booster='gbtree', random_state = 1, eval_metric='mlogloss', objective='binary:logistic', use_label_encoder=False)

# Train the model using train set
xgbc.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
xgbc_y_pred=xgbc.predict(X_test)

# Accuracy measures
print('Accuracy score of XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(xgbc_y_pred)), 3)*100)+'%')
print('Precision score of XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(xgbc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(xgbc_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of XGBC: 71.1%
Precision score of XGBC: 71.39999999999999%
Recall score of XGBC: 71.1%
F1 of XGBC: 71.0%


#### 6.1.7  <a class="anchor" id="6_1_7"></a> Naive Bayes

In [10]:
# Create a Naive Bayes model
gnb = GaussianNB()

# Train the model using train set
gnb.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
gnb_y_pred = gnb.predict(X_test)

# Accuracy measures
print('Accuracy score of GNB: ' + str(round(metrics.accuracy_score(Y_test, np.round(gnb_y_pred)), 3)*100)+'%')
print('Precision score of GNB: ' + str(round(metrics.precision_score(Y_test, np.round(gnb_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of GNB: ' + str(round(metrics.recall_score(Y_test, np.round(gnb_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of GNB: ' + str(round(metrics.f1_score(Y_test, np.round(gnb_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of GNB: 52.400000000000006%
Precision score of GNB: 56.99999999999999%
Recall score of GNB: 52.400000000000006%
F1 of GNB: 42.9%


#### 6.1.8  <a class="anchor" id="6_1_8"></a> Linear discriminant analysis

In [11]:
# Create a linear discriminant analysis model
lda = LinearDiscriminantAnalysis(n_components = 1)

# Train the model using train set
lda.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
lda_y_pred = lda.predict(X_test)

# Accuracy measures
print('Accuracy score of LDA: ' + str(round(metrics.accuracy_score(Y_test, np.round(lda_y_pred)), 3)*100)+'%')
print('Precision score of LDA: ' + str(round(metrics.precision_score(Y_test, np.round(lda_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of LDA: ' + str(round(metrics.recall_score(Y_test, np.round(lda_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of LDA: ' + str(round(metrics.f1_score(Y_test, np.round(lda_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of LDA: 68.10000000000001%
Precision score of LDA: 68.10000000000001%
Recall score of LDA: 68.10000000000001%
F1 of LDA: 68.10000000000001%


#### 6.1.9  <a class="anchor" id="6_1_9"></a> Quadratic discriminant analysis

In [12]:
# Create a QDA model
qda = QuadraticDiscriminantAnalysis()

# Train the model using train set
qda.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
qda_y_pred = qda.predict(X_test)

# Accuracy measures
print('Accuracy score of QDA: ' + str(round(metrics.accuracy_score(Y_test, np.round(qda_y_pred)), 3)*100)+'%')
print('Precision score of QDA: ' + str(round(metrics.precision_score(Y_test, np.round(qda_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of QDA: ' + str(round(metrics.recall_score(Y_test, np.round(qda_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of QDA: ' + str(round(metrics.f1_score(Y_test, np.round(qda_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of QDA: 54.400000000000006%
Precision score of QDA: 61.4%
Recall score of QDA: 54.400000000000006%
F1 of QDA: 46.2%


#### 6.1.10  <a class="anchor" id="6_1_10"></a> Mixture discriminant analysis

#### 6.1.11  <a class="anchor" id="6_1_11"></a> Regularized discriminant analysis

#### 6.1.12  <a class="anchor" id="6_1_12"></a> Flexible discriminant analysis

#### 6.1.13  <a class="anchor" id="6_1_13"></a> Neural network

#### 6.1.14  <a class="anchor" id="6_1_14"></a> Support vector machines

In [13]:
# Create an SVM Classifier model
svm = SVC(kernel='linear', random_state = 1, probability=True)

# Train the model using the train set
svm.fit(X_train, Y_train.values.ravel())

# Predict the response for test set
svm_y_pred = svm.predict(X_test)

# Accuracy measures
print('Accuracy score of SVM: ' + str(round(metrics.accuracy_score(Y_test, np.round(svm_y_pred)), 3)*100)+'%')
print('Precision score of SVM: ' + str(round(metrics.precision_score(Y_test, np.round(svm_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of SVM: ' + str(round(metrics.recall_score(Y_test, np.round(svm_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of SVM: ' + str(round(metrics.f1_score(Y_test, np.round(svm_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score of SVM: 68.30000000000001%
Precision score of SVM: 68.4%
Recall score of SVM: 68.30000000000001%
F1 of SVM: 68.30000000000001%


## Part 7: <a class="anchor" id="part7"></a> Fine-tuning

### 7.1  <a class="anchor" id="7_1"></a> XGBoost grid search

In [14]:
#URL: https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

# Defining parameter range
xgbc_grid = {'learning_rate':[0.1],
    'n_estimators':[1000],
    'max_depth':[4,5,6],
    'min_child_weight':[6,8,10,12],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'objective':['binary:logistic'],
    'nthread':[4],
    'scale_pos_weight':[1],
    'seed':[1],
    'eval_metric':['mlogloss']}

pprint(xgbc_grid)

{'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
 'eval_metric': ['mlogloss'],
 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
 'learning_rate': [0.1],
 'max_depth': [4, 5, 6],
 'min_child_weight': [6, 8, 10, 12],
 'n_estimators': [1000],
 'nthread': [4],
 'objective': ['binary:logistic'],
 'scale_pos_weight': [1],
 'seed': [1],
 'subsample': [0.6, 0.7, 0.8, 0.9]}


In [42]:
# Fitting the model for grid search
xgbc_tuned = GridSearchCV(XGBClassifier(), xgbc_grid, refit = True)
xgbc_tuned.fit(X_train, Y_train.values.ravel())

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, ...),
             param_grid={'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
                         'eval_metric': ['mlogloss'],
                         'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                         'learning_rate': [0.1], 'max_depth': [4, 5, 6],
                         'min_child_weight': [6, 8, 10, 12],
                         'n_estimators': [1000], 'nthread': [4],
                         'objective': ['binary:logistic'],
                         'scale_pos_weight': [1], 'seed': [1],
                         'subsample': [0.6, 0.7, 0.8, 0.9]})

In [43]:
# Print best parameter after tuning
print(xgbc_tuned.best_params_)
 
# Print how our model looks after hyper-parameter tuning
print(xgbc_tuned.best_estimator_)

{'colsample_bytree': 0.6, 'eval_metric': 'mlogloss', 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 10, 'n_estimators': 1000, 'nthread': 4, 'objective': 'binary:logistic', 'scale_pos_weight': 1, 'seed': 1, 'subsample': 0.8}
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              predictor='auto', random_state=1, reg_alpha=0, ...)


In [17]:
# Create a tuned XGBC model
xgbc_tuned = XGBClassifier(colsample_bytree=0.6, eval_metric='mlogloss',
gamma=0.2, learning_rate=0.1, max_depth=4, min_child_weight=10, n_estimators=1000,
nthread=4, objective='binary:logistic', scale_pos_weight = 1, seed=1, subsample=0.8)
xgbc_tuned.fit(X_train, Y_train.values.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=10, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              predictor='auto', random_state=1, reg_alpha=0, ...)

In [18]:
# Base model results
xgbc_base_y_pred = xgbc.predict(X_test)
xgbc_base_accuracy = round(metrics.accuracy_score(Y_test, np.round(xgbc_base_y_pred)), 3)*100
print('Accuracy of base XGBC is ' + str(xgbc_base_accuracy)+'%')

# Tuned model results
xgbc_tuned_y_pred = xgbc_tuned.predict(X_test)
xgbc_tuned_accuracy = round(metrics.accuracy_score(Y_test, xgbc_tuned_y_pred), 3)*100
print('Accuracy of tuned XGBC is ' + str(xgbc_tuned_accuracy)+'%')

# Comparison
print('Improvement of {:0.1f}%'.format(100 * (xgbc_tuned_accuracy - xgbc_base_accuracy) / xgbc_base_accuracy))

Accuracy of base XGBC is 71.1%
Accuracy of tuned XGBC is 69.3%
Improvement of -2.5%


In [19]:
# Rest of the measures
print('Precision score of tuned XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(xgbc_tuned_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('Recall score of tuned XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(xgbc_tuned_y_pred), average='weighted'), 3)*100)+'%')
print('F1 of tuned XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(xgbc_tuned_y_pred), average='weighted'), 3)*100)+'%')

Precision score of tuned XGBC: 69.3%
Recall score of tuned XGBC: 69.3%
F1 of tuned XGBC: 69.3%


### 7.3  <a class="anchor" id="7_2"></a> SVM grid search

In [24]:
# Look at parameters used by our current SVM model
print('Parameters currently in use:\n')
pprint(svm.get_params())

Parameters currently in use:

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': True,
 'random_state': 1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [25]:
#URL: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/

# Defining parameter range
svm_grid = {'C': [0.1, 1, 10],
            'gamma': [1, 0.1, 0.01],
            'kernel': ['linear']}
 
# Fitting the model for grid search
svm_tuned = GridSearchCV(SVC(), svm_grid, refit = True) 
svm_tuned.fit(X_train, Y_train.values.ravel())

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']})

In [26]:
# Print best parameter after tuning
print(svm_tuned.best_params_)
 
# Print how our model looks after hyper-parameter tuning
print(svm_tuned.best_estimator_)

{'C': 10, 'gamma': 1, 'kernel': 'linear'}
SVC(C=10, gamma=1, kernel='linear')


In [27]:
# Base model results
svm_base_y_pred = svm.predict(X_test)
svm_base_accuracy = round(metrics.accuracy_score(Y_test, svm_base_y_pred), 3)*100
print('Accuracy of base SVM is ' + str(svm_base_accuracy)+'%')

# Tuned model results
svm_tuned_y_pred = svm_tuned.predict(X_test)
svm_tuned_accuracy = round(metrics.accuracy_score(Y_test, svm_tuned_y_pred), 3)*100
print('Accuracy of tuned SVM is ' + str(svm_tuned_accuracy)+'%')

print('Improvement of {:0.2f}%'.format(100 * (svm_tuned_accuracy - svm_base_accuracy) / svm_base_accuracy))

Accuracy of base SVM is 68.30000000000001%
Accuracy of tuned SVM is 68.4%
Improvement of 0.15%


## Part 8: <a class="anchor" id="part8"></a> Ensemble learning

### 8.1  <a class="anchor" id="8_1"></a> Voting classifier

In [28]:
soft_voting = VotingClassifier(
    estimators=[('xgbc', xgbc), ('rfc', rfc)],
    voting='soft')

soft_voting.fit(X_train, Y_train.values.ravel())

VotingClassifier(estimators=[('xgbc',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='mlogloss', gamma=0,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_type=None,
                                            interaction_constraints='',
                                            learning_rate=0.05, max_bin=256,
                                            max_cat_to_onehot=4,
                                            max_delta_step=0, max_depth=6,
                                            max_leaves=0, min_child_weight=1,
                                            missing=nan,
                                            monotone_constraints='()',
                                            n_estimators=100, n_jobs=0,
                                            num_parallel_tree=1,
                                            predictor='auto', random_state=1,
                                            reg_alpha=0, reg_lambda=1, ...)),
                             ('rfc', RandomForestClassifier(random_state=1))],
                 voting='soft')

### 8.2  <a class="anchor" id="8_2"></a> Stacking

#### 8.2.1  <a class="anchor" id="8_2_1"></a> Top 9 models

In [ ]:
#URL: https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/

def get_stacking():
	# Define the base models
	level9 = list()
	level9.append(('dtc', dtc))
	level9.append(('knn', knn_100))
	level9.append(('rdg', rdg))
	level9.append(('lda', lda))
	level9.append(('log', log))
	level9.append(('svm_t', svm_tuned))
	level9.append(('rfc', rfc))
	level9.append(('sv', soft_voting))
	level9.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on base XGBC
	model = StackingClassifier(estimators=level9, final_estimator=xgbc, cv=5)
	return model

In [ ]:
# Define the base models separately
level9 = list()
level9.append(('gnb', gnb))
level9.append(('qda', qda))
level9.append(('lda', lda))
level9.append(('log', log))
level9.append(('knn', knn_7))
level9.append(('dtc', dtc))
level9.append(('hard voting', hard_voting))
level9.append(('rfc tuned', rfc_tuned))
level9.append(('xgbc', xgbc))
level9.append(('stacking', get_stacking()))

In [ ]:
# Define the stacking ensemble learnt on base XGBC
stack9_xgbc = StackingClassifier(estimators=level9, final_estimator=xgbc, cv=5)

# Fit the model on all available data
stack9_xgbc = stack9_xgbc.fit(X, Y.values.ravel())

# Predict the response for test set
stack9_xgbc_y_pred = stack9_xgbc.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 9 models: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack9_xgbc_y_pred)), 3)*100)+'%')
print('Recall score  with 9 models: ' + str(round(metrics.recall_score(Y_test, np.round(stack9_xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score  with 9 models: ' + str(round(metrics.precision_score(Y_test, np.round(stack9_xgbc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 9 models: ' + str(round(metrics.f1_score(Y_test, np.round(stack9_xgbc_y_pred), average='weighted'), 3)*100)+'%')

#### 8.2.2  <a class="anchor" id="8_2_2"></a> Top 8 models

In [ ]:
def get_stacking():
	# Define the base models
	level8 = list()
	level8.append(('qda', qda))
	level8.append(('lda', lda))
	level8.append(('log', log))
	level8.append(('knn', knn_7))
	level8.append(('dtc', dtc))
	level8.append(('hard voting', hard_voting))
	level8.append(('rfc tuned', rfc_tuned))
	level8.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on hard voting classifier
	model = StackingClassifier(estimators=level8, final_estimator=hard_voting, cv=5)
	return model

In [ ]:
# Define the base models seperately
level8 = list()
level8.append(('qda', qda))
level8.append(('lda', lda))
level8.append(('log', log))
level8.append(('knn', knn_7))
level8.append(('dtc', dtc))
level8.append(('hard voting', hard_voting))
level8.append(('rfc tuned', rfc_tuned))
level8.append(('xgbc', xgbc))
level8.append(('stacking', get_stacking()))

In [ ]:
# Define the stacking ensemble learnt on hard voting classifier
#stack8_hv = StackingClassifier(estimators=level8, final_estimator=hard_voting, cv=5)

# Fit the model on all available data
#stack8_hv = stack8_hv.fit(X, Y.values.ravel())

# Predict the response for test set
#stack8_hv_y_pred = stack8_hv.predict(X_test)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
np.all(np.isfinite(X))

In [ ]:
np.any(np.isnan(X))

In [ ]:
# Accuracy measures
print('Accuracy score with 8 models learnt on hard voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack8_hv_y_pred)), 3)*100)+'%')
print('Recall score with 8 models learnt on hard voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack8_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 8 models learnt on hard voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack8_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 8 models learnt on hard voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack8_hv_y_pred), average='weighted'), 3)*100)+'%')

#### 8.2.3  <a class="anchor" id="8_2_3"></a> Top 7 models

In [ ]:
#URL: https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/

# get a stacking ensemble of models
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('lda', lda))
	level0.append(('log', log))
	level0.append(('knn', knn_7))
	level0.append(('dtc', dtc))
	level0.append(('hard voting', hard_voting))
	level0.append(('rfc tuned', rfc_tuned))
	level0.append(('xgbc', xgbc))

	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=hard_voting, cv=5)
	return model

# get a list of models to evaluate
def get_models():
	models = dict()
	models['LDA'] = lda
	models['LOG'] = log
	models['KNN'] = knn_7
	models['DTC'] = dtc
	models['HD'] = hard_voting
	models['RFC_T'] = rfc_tuned
	models['XGBC'] = xgbc
	models['STACKING'] = get_stacking()
	return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, Y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, Y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# Define the base models separately
level7 = list()
level7.append(('lda', lda))
level7.append(('log', log))
level7.append(('knn', knn_7))
level7.append(('dtc', dtc))
level7.append(('hard voting', hard_voting))
level7.append(('rfc tuned', rfc_tuned))
level7.append(('xgbc', xgbc))
level7.append(('stacking', get_stacking()))

In [ ]:
def get_stacking():
	# Define the base models
	level7 = list()
	level7.append(('lda', lda))
	level7.append(('log', log))
	level7.append(('knn', knn_7))
	level7.append(('dtc', dtc))
	level7.append(('hard voting', hard_voting))
	level7.append(('rfc tuned', rfc_tuned))
	level7.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on base XGBC
	model = StackingClassifier(estimators=level7, final_estimator=xgbc, cv=5)
	return model

In [ ]:
# Define the stacking ensemble learnt on base XGBC
stack7_xgbc = StackingClassifier(estimators=level7, final_estimator=xgbc, cv=5)

# Fit the model on all available data
stack7_xgbc = stack7_xgbc.fit(X, Y.values.ravel())

# Predict the response for test set
stack7_xgbc_y_pred = stack7_xgbc.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 7 models learnt on base XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack7_xgbc_y_pred)), 3)*100)+'%')
print('Recall score with 7 models learnt on base XGBC:' + str(round(metrics.recall_score(Y_test, np.round(stack7_xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 7 models learnt on base XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack7_xgbc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 7 models learnt on base XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack7_xgbc_y_pred), average='weighted'), 3)*100)+'%')

In [ ]:
def get_stacking():
	# Define the base models
	level7 = list()
	level7.append(('lda', lda))
	level7.append(('log', log))
	level7.append(('knn', knn_7))
	level7.append(('dtc', dtc))
	level7.append(('hard voting', hard_voting))
	level7.append(('rfc tuned', rfc_tuned))
	level7.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on base XGBC
	model = StackingClassifier(estimators=level7, final_estimator=hard_voting, cv=5)
	return model

In [ ]:
# Define the stacking ensemble learnt on hard voting classifier
stack7_hv = StackingClassifier(estimators=level7, final_estimator=hard_voting, cv=5)

# Fit the model on all available data
stack7_hv = stack7_hv.fit(X, Y.values.ravel())

# Predict the response for test set
stack7_hv_y_pred = stack7_hv.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 7 models based on hard voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack7_hv_y_pred)), 3)*100)+'%')
print('Recall score with 7 models based on hard voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack7_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 7 models based on hard voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack7_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 7 models based on hard voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack7_hv_y_pred), average='weighted'), 3)*100)+'%')

#### 8.2.4  <a class="anchor" id="8_2_4"></a> Top 6 models

In [ ]:
# Define the base models
level6 = list()
level6.append(('log', log))
level6.append(('knn', knn_7))
level6.append(('dtc', dtc))
level6.append(('hard voting', hard_voting))
level6.append(('rfc tuned', rfc_tuned))
level6.append(('xgbc', xgbc))
level6.append(('stacking', get_stacking()))

In [ ]:
def get_stacking():
	# Define the base models
	level0 = list()
	level0.append(('log', log))
	level0.append(('knn', knn_7))
	level0.append(('dtc', dtc))
	level0.append(('hard voting', hard_voting))
	level0.append(('rfc tuned', rfc_tuned))
	level0.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on hard voting classifier
	model = StackingClassifier(estimators=level0, final_estimator=hard_voting, cv=5)
	return model

In [ ]:
# Define the stacking ensemble learnt on hard voting classifier
stack6_hv = StackingClassifier(estimators=level6, final_estimator=hard_voting, cv=5)

# Fit the model on all available data
stack6_hv = stack6_hv.fit(X, Y.values.ravel())

# Predict the response for test set
stack6_hv_y_pred = stack6_hv.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 6 models learnt on hard voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack6_hv_y_pred)), 3)*100)+'%')
print('Recall score with 6 models learnt on hard voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack6_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 6 models learnt on hard voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack6_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 6 models learnt on hard voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack6_hv_y_pred), average='weighted'), 3)*100)+'%')

In [ ]:
def get_stacking():
	# Define the base models
	level0 = list()
	level0.append(('log', log))
	level0.append(('knn', knn_7))
	level0.append(('dtc', dtc))
	level0.append(('hard voting', hard_voting))
	level0.append(('rfc tuned', rfc_tuned))
	level0.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on base Gaussian classifier
	model = StackingClassifier(estimators=level0, final_estimator=xgbc, cv=5)
	return model

In [ ]:
# Define the stacking ensemble learnt on base Gaussian classifier
stack6_xgbc = StackingClassifier(estimators=level6, final_estimator=xgbc, cv=5)

# Fit the model on all available data
stack6_xgbc = stack6_xgbc.fit(X, Y.values.ravel())

# Predict the response for test set
stack6_xgbc_y_pred = stack6_xgbc.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 6 models learnt on base XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack6_xgbc_y_pred)), 3)*100)+'%')
print('Recall score with 6 models learnt on base XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack6_xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 6 models learnt on base XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack6_xgbc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 6 models learnt on base XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack6_xgbc_y_pred), average='weighted'), 3)*100)+'%')

In [ ]:
# Get a list of models to evaluate
def get_models():
	models = dict()
	models['LOG'] = log
	models['KNN'] = knn_7
	models['DTC'] = dtc
	models['HD'] = hard_voting
	models['RFC_T'] = rfc_tuned
	models['XGBC'] = xgbc
	models['STACKING'] = get_stacking()
	return models

# Evaluate a given model using cross-validation
def evaluate_model(model, X, Y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

# Get the models to evaluate
models = get_models()

# Evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, Y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

# Plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

#### 8.2.5  <a class="anchor" id="8_2_4"></a> Top 5 models

In [ ]:
# Define the base models separately
level5 = list()
level5.append(('knn', knn_7))
level5.append(('dtc', dtc))
level5.append(('hard voting', hard_voting))
level5.append(('rfc tuned', rfc_tuned))
level5.append(('xgbc', xgbc))
level5.append(('stacking', get_stacking()))

In [ ]:
def get_stacking():
	# Define the base models
	level5 = list()
	level5.append(('knn', knn_7))
	level5.append(('dtc', dtc))
	level5.append(('hard voting', hard_voting))
	level5.append(('rfc tuned', rfc_tuned))
	level5.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on hard voting classifier
	model = StackingClassifier(estimators=level5, final_estimator=hard_voting, cv=5)
	return model

In [ ]:
# Define the stacking ensemble learnt on hard voting classifier
stack5_hv = StackingClassifier(estimators=level5, final_estimator=hard_voting, cv=5)

# Fit the model on all available data
stack5_hv = stack5_hv.fit(X, Y.values.ravel())

# Predict the response for test set
stack5_hv_y_pred = stack5_hv.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 5 models learnt on hard voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack5_hv_y_pred)), 3)*100)+'%')
print('Recall score with 5 models learnt on hard voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack5_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 5 models learnt on hard voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack5_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 5 models learnt on hard voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack5_hv_y_pred), average='weighted'), 3)*100)+'%')

In [ ]:
def get_stacking():
	# Define the base models
	level5 = list()
	level5.append(('knn', knn_7))
	level5.append(('dtc', dtc))
	level5.append(('hard voting', hard_voting))
	level5.append(('rfc tuned', rfc_tuned))
	level5.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on base Gaussian classifier
	model = StackingClassifier(estimators=level5, final_estimator=xgbc, cv=5)
	return model

In [ ]:
# Define the stacking ensemble learnt on base Gaussian classifier
stack5_xgbc = StackingClassifier(estimators=level5, final_estimator=xgbc, cv=5)

# Fit the model on all available data
stack5_xgbc = stack5_xgbc.fit(X, Y.values.ravel())

# Predict the response for test set
stack5_xgbc_y_pred = stack5_xgbc.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 5 models learnt on base XGBC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack5_xgbc_y_pred)), 3)*100)+'%')
print('Recall score with 5 models learnt on base XGBC: ' + str(round(metrics.recall_score(Y_test, np.round(stack5_xgbc_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 5 models learnt on base XGBC: ' + str(round(metrics.precision_score(Y_test, np.round(stack5_xgbc_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 5 models learnt on base XGBC: ' + str(round(metrics.f1_score(Y_test, np.round(stack5_xgbc_y_pred), average='weighted'), 3)*100)+'%')

#### 8.2.6  <a class="anchor" id="8_2_6"></a> Top 4 models

In [70]:
def get_stacking():
	# Define the base models
	level4 = list()
	level4.append(('svm_t', svm_tuned))
	level4.append(('rfc', rfc))
	level4.append(('sv', soft_voting))
	level4.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on hard voting classifier
	model = StackingClassifier(estimators=level4, final_estimator=soft_voting, cv=5)
	return model

In [69]:
# Define the base models separately
level4 = list()
level4.append(('svm_t', svm_tuned))
level4.append(('rfc', rfc))
level4.append(('sv', soft_voting))
level4.append(('xgbc', xgbc))
level4.append(('stacking', get_stacking()))

In [71]:
# Define the stacking ensemble learnt on hard voting classifier
stack4_sv = StackingClassifier(estimators=level4, final_estimator=soft_voting, cv=5)

# Fit the model on all available data
stack4_sv = stack4_sv.fit(X, Y.values.ravel())

# Predict the response for test set
stack4_sv_y_pred = stack4_sv.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 4 models learnt on soft voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack4_sv_y_pred)), 3)*100)+'%')
print('Recall score with 4 models learnt on soft voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack4_sv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 4 models learnt on soft voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack4_sv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 4 models learnt on soft voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack4_sv_y_pred), average='weighted'), 3)*100)+'%')

#### 8.2.7  <a class="anchor" id="8_2_7"></a> Top 3 models

In [63]:
def get_stacking():
	# Define the base models
	level3 = list()
	level3.append(('rfc', rfc))
	level3.append(('sv', soft_voting))
	level3.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on hard voting classifier
	model = StackingClassifier(estimators=level3, final_estimator=soft_voting, cv=5)
	return model

In [64]:
# Define the base models separately
level3 = list()
level3.append(('rfc', rfc))
level3.append(('sv', soft_voting))
level3.append(('xgbc', xgbc))
level3.append(('stacking', get_stacking()))

In [65]:
# Define the stacking ensemble learnt on hard voting classifier
stack3_sv = StackingClassifier(estimators=level3, final_estimator=soft_voting, cv=5)

# Fit the model on all available data
stack3_sv = stack3_sv.fit(X, Y.values.ravel())

# Predict the response for test set
stack3_sv_y_pred = stack3_sv.predict(X_test)

In [66]:
# Accuracy measures
print('Accuracy score with 3 models learnt on soft voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack3_sv_y_pred)), 3)*100)+'%')
print('Recall score with 3 models learnt on soft voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack3_sv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 3 models learnt on soft voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack3_sv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 3 models learnt on soft voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack3_sv_y_pred), average='weighted'), 3)*100)+'%')

Accuracy score with 3 models learnt on soft voting classifier: 82.3%
Recall score with 3 models learnt on soft voting classifier: 82.3%
Precision score with 3 models learnt on soft voting classifier: 82.3%
F1 score with 3 models learnt on soft voting classifier: 82.3%


In [ ]:
def get_stacking():
	# Define the base models
	level3 = list()
	level3.append(('hard voting', hard_voting))
	level3.append(('rfc tuned', rfc_tuned))
	level3.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on tuned random forest classifier
	model = StackingClassifier(estimators=level3, final_estimator=rfc_tuned, cv=5)
	return model

In [ ]:
# Define the base models separately
level3 = list()
level3.append(('hard voting', hard_voting))
level3.append(('rfc tuned', rfc_tuned))
level3.append(('xgbc', xgbc))
level3.append(('stacking', get_stacking()))

In [ ]:
# Define the stacking ensemble learnt on tuned random forest classifier
stack3_rfc_t = StackingClassifier(estimators=level3, final_estimator=rfc_tuned, cv=5)

# Fit the model on all available data
stack3_rfc_t = stack3_rfc_t.fit(X, Y.values.ravel())

# Predict the response for test set
stack3_rfc_t_y_pred = stack3_rfc_t.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 3 models learnt on tuned RFC: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack3_rfc_t_y_pred)), 3)*100)+'%')
print('Recall score with 3 models learnt on tuned RFC: ' + str(round(metrics.recall_score(Y_test, np.round(stack3_rfc_t_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 3 models learnt on tuned RFC: ' + str(round(metrics.precision_score(Y_test, np.round(stack3_rfc_t_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 3 models learnt on tuned RFC: ' + str(round(metrics.f1_score(Y_test, np.round(stack3_rfc_t_y_pred), average='weighted'), 3)*100)+'%')

#### 8.2.8  <a class="anchor" id="8_2_8"></a> Top 2 models

In [ ]:
def get_stacking():
	# Define the base models
	level2 = list()
	level2.append(('hard voting', hard_voting))
	level2.append(('xgbc', xgbc))

	# Define the stacking ensemble learnt on hard voting classifier
	model = StackingClassifier(estimators=level2, final_estimator=hard_voting, cv=5)
	return model

In [ ]:
# Define the base models separately
level2 = list()
level2.append(('hard voting', hard_voting))
level2.append(('xgbc', xgbc))
level2.append(('stacking', get_stacking()))

In [ ]:
# Define the stacking ensemble learnt on hard voting classifier
stack2_hv = StackingClassifier(estimators=level2, final_estimator=hard_voting, cv=5)

# Fit the model on all available data
stack2_hv = stack2_hv.fit(X, Y.values.ravel())

# Predict the response for test set
stack2_hv_y_pred = stack2_hv.predict(X_test)

In [ ]:
# Accuracy measures
print('Accuracy score with 2 models learnt on hard voting classifier: ' + str(round(metrics.accuracy_score(Y_test, np.round(stack2_hv_y_pred)), 3)*100)+'%')
print('Recall score with 2 models learnt on hard voting classifier: ' + str(round(metrics.recall_score(Y_test, np.round(stack2_hv_y_pred), average='weighted'), 3)*100)+'%')
print('Precision score with 2 models learnt on hard voting classifier: ' + str(round(metrics.precision_score(Y_test, np.round(stack2_hv_y_pred), average='weighted', zero_division=1), 3)*100)+'%')
print('F1 score with 2 models learnt on hard voting classifier: ' + str(round(metrics.f1_score(Y_test, np.round(stack2_hv_y_pred), average='weighted'), 3)*100)+'%')

## Part 9: <a class="anchor" id="part9"></a> Evaluation of the final model

### 9.1  <a class="anchor" id="9_1"></a> Confusion matrix

In [ ]:
# URL: https://pythonawesome.com/multi-class-confusion-matrix-library-in-python/#:~:text=PyCM%20is%20a%20multi-class%20confusion%20matrix%20library%20written,that%20supports%20most%20classes%20and%20overall%20statistics%20parameters.

y_actu = Y_test['Rec_Class'].to_numpy()
y_pred = stack3_hv_y_pred
cm = ConfusionMatrix(actual_vector=y_actu, predict_vector=y_pred)
cm.classes
print(cm)

* AUC of class 0 is 0.52 which is poor.
* AUC of class 1 is 0.79 which is good.
* AUC of class 2 is 0.82 which is very good.
<br />
<br />
* F1 of class 0 is 0.09 which is poor.
* F1 of class 1 is 0.78 which is good.
* F1 of class 2 is 0.82 which is very good.
<br />
<br />
Class 0 - low recoverability/unrecoverable projects
Class 1 - average recoverability
Class 2 - highly recoverable projects

In [ ]:
#URL: https://www.stackvidhya.com/plot-confusion-matrix-in-python-and-why/#:~:text=You%20can%20create%20the%20confusion%20matrix%20using%20the,Positives%2C%20False%20Negatives%2C%20and%20True%20negatives.%20%2A%2A%20Snippet%2A%2A

plt.subplots(figsize = (8,7))
final_matrix = confusion_matrix(Y_test, stack3_hv_y_pred)
ax = sns.heatmap(final_matrix/np.sum(final_matrix), annot=True, fmt='.1%', linewidths=.2, cmap='Greens')

ax.set_title('Confusion Matrix of the Final Recoverability Model\n', fontsize = 14);
ax.set_xlabel('\nPredicted recoverability class')
ax.set_ylabel('Actual recoverability class');
ax.xaxis.set_ticklabels(['low','average', 'high'])
ax.yaxis.set_ticklabels(['low','average', 'high'])
plt.savefig('figures/rec/Confusion Matrix of the Final Recoverability Model')

### 9.2  <a class="anchor" id="9_2"></a> Cost matrix

In [ ]:
# URL: https://imbalanced-ensemble.readthedocs.io/en/latest/auto_examples/classification/plot_cost_matrix.html

init_kwargs = {
    'n_estimators': 5,
    'random_state': 1,
}
adacost_clf = imbens.ensemble.AdaCostClassifier(**init_kwargs)

cost_matrices = {}

def plot_cost_matrix(cost_matrix, title:str, **kwargs):
    ax = sns.heatmap(data=cost_matrix, **kwargs)
    ax.set_ylabel('Predicted class')
    ax.set_xlabel('Ground truth')
    ax.set_title(title)

adacost_clf.fit(X_train, Y_train.values.ravel())

In [ ]:
# Default cost matrix
plt.subplots(figsize = (8,7))
title = 'Misclassification Cost Matrix\n(by inverse class frequency)'
cost_matrices[title] = adacost_clf.cost_matrix_
plot_cost_matrix(adacost_clf.cost_matrix_, title, annot=True, cmap='YlOrRd', vmax=17, linewidths=.2)
plt.savefig('figures/rec/Misclassification Cost Matrix')

In this cost matrix:
* correct classification of highly recoverable projects gives a +1 benefit,
* misclassification of a highly recoverable project as average project has nearly the same impact 0f 0.94 units,
* whilst mistaking a highly recoverable project for an unrecoverable one is neutral (+0.059);
<br />
<br />
* correct classification of average projects gives a +1 benefit,
* misclassification of an average project as highly recoverable project has nearly the same impact of +1.1 units,
* mistaking an average project for an unrecoverable one is also neutral (+0.063);
<br />
<br />
* correct classification of unrecoverable projects gives a +1 benefit,
* misclassification of an unrecoverable project for an average one brings 16 units of impact,
* mistaking an unrecoverable project for highly recoverable has the most drastic impact on prediction cost with 17 units of impact.

### 9.3  <a class="anchor" id="9_3"></a> Receiver Operating Characteristic (ROC) curve

In [ ]:
stack3_hv_param = stack3_hv.get_params()
stack3_hv_param['num_class'] = 3
stack3_hv_param

In [ ]:
ovrc_param = OVRC.get_params()
ovrc_param['num_class'] = 3
ovrc_param

In [ ]:
#URL: https://stackoverflow.com/questions/45332410/roc-for-multiclass-classification

# Binarize the output
Y = label_binarize(Y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Learn to predict each class against the other
FINAL_MODEL = stack3_hv
OVRC = OneVsRestClassifier(FINAL_MODEL)
OVRC.fit(X_train, Y_train)

Y = label_binarize(Y, classes=[0,1,2])

# Shuffle and split training and test sets
X_train, X_test, Y_train, Y_test =\
    train_test_split(X, Y, test_size=0.2, random_state=1)

# Classifier
Y_score = OVRC.fit(X_train, Y_train).predict(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], Y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('false positive rate')
    plt.ylabel('true Positive Rate')
    plt.title('ROC curve class {}'.format(i))
    plt.legend(loc='lower right')
    plt.show()
    plt.savefig('figures/rec/ROC curve for each recoverability class.png')

In [ ]:
#URL: https://stackoverflow.com/questions/56090541/how-to-plot-precision-and-recall-of-multiclass-classifier

fpr = dict()
tpr = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i],
                                  Y_score[:, i])
    plt.plot(fpr[i], tpr[i], lw=2, label='class {}'.format(i))

plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend(loc='best')
plt.title('ROC curve for all classes')
plt.show()
plt.savefig('figures/rec/ROC curve for all recoverability classes')

In [ ]:
#URL: https://stackoverflow.com/questions/45332410/roc-for-multiclass-classification

# Binarize the output
Y = label_binarize(Y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Learn to predict each class against the other
FINAL_MODEL = xgbc
OVRC = OneVsRestClassifier(FINAL_MODEL)
OVRC.fit(X_train, Y_train)

Y = label_binarize(Y, classes=[0,1,2])
n_classes = 3

# shuffle and split training and test sets
X_train, X_test, Y_train, Y_test =\
    train_test_split(X, Y, test_size=0.2, random_state=1)

# classifier
Y_score = OVRC.fit(X_train, Y_train).predict(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], Y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('false positive rate')
    plt.ylabel('true Positive Rate')
    plt.title('ROC curve class {}'.format(i))
    plt.legend(loc='lower right')
    plt.show()
    plt.savefig('figures/rec/ROC curve for each recoverability class')

In [ ]:
#URL: https://stackoverflow.com/questions/56090541/how-to-plot-precision-and-recall-of-multiclass-classifier

fpr = dict()
tpr = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i],
                                  Y_score[:, i])
    plt.plot(fpr[i], tpr[i], lw=2, label='class {}'.format(i))

plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend(loc='best')
plt.title('ROC curve for all classes')
plt.show()
plt.savefig('figures/rec/ROC curve for all recoverability classes')

### 9.4  <a class="anchor" id="9_4"></a> Precision-recall curve

In [ ]:
#URL: https://stackoverflow.com/questions/56090541/how-to-plot-precision-and-recall-of-multiclass-classifier

precision = dict()
recall = dict()
for i in range(3):
    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        Y_score[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc='best')
plt.title('Precision vs. Recall Curve')
plt.show()
plt.savefig('figures/rec/Precision-recall curve for all recoverability classes')

### 9.5  <a class="anchor" id="9_5"></a> Feature importance

In [ ]:
xgbc.feature_names = X.columns
final_feature_importance = pd.DataFrame({'Feature': xgbc.feature_names,'Importance':xgbc.feature_importances_})
final_feature_importance = final_feature_importance.sort_values(by = ['Importance'], ascending = False)

In [ ]:
# Plotting a bar plot for feature importance
fig, ax = plt.subplots(figsize = (16,10))
sns.barplot(final_feature_importance['Importance'], final_feature_importance['Feature'], color = 'green')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.xticks(rotation = 'horizontal')
plt.title('Feature Importance')
plt.savefig('figures/rec/Final Feature Importance')